In [1]:
import torch

from helpers.model.util import select_device
from helpers.experiment.experiment import Experiment
from helpers.experiment.configs import (
    Config_Experiment_Images,
    Config_Experiment_Deep_Sets,
    Config_Experiment_Event_by_Event
)
from helpers.data.dset.config import Config_Dataset
from helpers.data.dset.constants import (
    Names_Datasets,
    Names_Levels,
    Names_q_Squared_Vetos,
    Names_Splits,
    Names_Variables,
    Nums_Events_Per_Set
)
from helpers.model.config import Config_Model
from helpers.model.constants import Names_Models
from helpers.plot.util import setup_high_quality_mpl_params


setup_high_quality_mpl_params()

path_dir_plots = "../../state/new_physics/plots"


device = select_device()


experiment = Experiment(
    path_dir_plots=path_dir_plots,
    device=device,
)


Device:  cuda


Images Method

In [2]:

config_experiment_images = Config_Experiment_Images()

Train

In [ ]:
for level in (Names_Levels().detector_and_background,):

    for num_events_per_set in Nums_Events_Per_Set().tuple_:

        experiment.train(
            config_model=config_experiment_images.get_config_model(
                level=level, 
                num_events_per_set=num_events_per_set
            ),
            config_dset_eval=config_experiment_images.get_config_dset(
                level=level, 
                num_events_per_set=num_events_per_set, 
                kind="eval"
            ),
            generate_dsets=True,
        )

Eval

In [3]:
for level in Names_Levels().tuple_:

    for num_events_per_set in Nums_Events_Per_Set().tuple_:

        experiment.evaluate(
            config_model=config_experiment_images.get_config_model(
                level=level, 
                num_events_per_set=num_events_per_set
            ), 
            config_dset_eval=config_experiment_images.get_config_dset(
                level=level, 
                num_events_per_set=num_events_per_set, 
                kind="eval"
            ),
            config_dset_eval_sens=config_experiment_images.get_config_dset(
                level=level, 
                num_events_per_set=num_events_per_set, 
                kind="eval_sens"
            ),
            generate_dsets=False, 
        )

Loaded tensor of shape: torch.Size([2000, 1, 10, 10, 10]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\70000_eval_sens_features.pt
Loaded tensor of shape: torch.Size([2000]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\70000_eval_sens_labels.pt
Loaded dataset: images
Loaded tensor of shape: torch.Size([2200, 1, 10, 10, 10]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\70000_eval_features.pt
Loaded tensor of shape: torch.Size([2200]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\70000_eval_labels.pt
Loaded dataset: images


c:\Users\tetha\Desktop\btokstll\logic\scripts\helpers\result\table.py:37: PerformanceWarning: indexing past lexsort depth may impact performance.
  self.table.loc[


Loaded tensor of shape: torch.Size([2000, 1, 10, 10, 10]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\24000_eval_sens_features.pt
Loaded tensor of shape: torch.Size([2000]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\24000_eval_sens_labels.pt
Loaded dataset: images
Loaded tensor of shape: torch.Size([2200, 1, 10, 10, 10]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\24000_eval_features.pt
Loaded tensor of shape: torch.Size([2200]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\24000_eval_labels.pt
Loaded dataset: images


c:\Users\tetha\Desktop\btokstll\logic\scripts\helpers\result\table.py:37: PerformanceWarning: indexing past lexsort depth may impact performance.
  self.table.loc[


Loaded tensor of shape: torch.Size([2000, 1, 10, 10, 10]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\6000_eval_sens_features.pt
Loaded tensor of shape: torch.Size([2000]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\6000_eval_sens_labels.pt
Loaded dataset: images
Loaded tensor of shape: torch.Size([2200, 1, 10, 10, 10]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\6000_eval_features.pt
Loaded tensor of shape: torch.Size([2200]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\6000_eval_labels.pt
Loaded dataset: images


...

c:\Users\tetha\Desktop\btokstll\logic\scripts\helpers\result\table.py:37: PerformanceWarning: indexing past lexsort depth may impact performance.
  self.table.loc[


Loaded tensor of shape: torch.Size([2000, 1, 10, 10, 10]) from: ..\..\state\new_physics\data\processed\images_det_bkg_q2v_loose\24000_eval_sens_features.pt
Loaded tensor of shape: torch.Size([2000]) from: ..\..\state\new_physics\data\processed\images_det_bkg_q2v_loose\24000_eval_sens_labels.pt
Loaded dataset: images
Loaded tensor of shape: torch.Size([2200, 1, 10, 10, 10]) from: ..\..\state\new_physics\data\processed\images_det_bkg_q2v_loose\24000_eval_features.pt
Loaded tensor of shape: torch.Size([2200]) from: ..\..\state\new_physics\data\processed\images_det_bkg_q2v_loose\24000_eval_labels.pt
Loaded dataset: images


c:\Users\tetha\Desktop\btokstll\logic\scripts\helpers\result\table.py:37: PerformanceWarning: indexing past lexsort depth may impact performance.
  self.table.loc[


Loaded tensor of shape: torch.Size([2000, 1, 10, 10, 10]) from: ..\..\state\new_physics\data\processed\images_det_bkg_q2v_loose\6000_eval_sens_features.pt
Loaded tensor of shape: torch.Size([2000]) from: ..\..\state\new_physics\data\processed\images_det_bkg_q2v_loose\6000_eval_sens_labels.pt
Loaded dataset: images
Loaded tensor of shape: torch.Size([2200, 1, 10, 10, 10]) from: ..\..\state\new_physics\data\processed\images_det_bkg_q2v_loose\6000_eval_features.pt
Loaded tensor of shape: torch.Size([2200]) from: ..\..\state\new_physics\data\processed\images_det_bkg_q2v_loose\6000_eval_labels.pt
Loaded dataset: images


c:\Users\tetha\Desktop\btokstll\logic\scripts\helpers\result\table.py:37: PerformanceWarning: indexing past lexsort depth may impact performance.
  self.table.loc[


Deep Sets Method

In [4]:
config_experiment_deep_sets = Config_Experiment_Deep_Sets()

Train

In [ ]:
for level in (Names_Levels().detector_and_background,):

    for num_events_per_set in Nums_Events_Per_Set().tuple_:

        experiment.train(
            config_model=config_experiment_deep_sets.get_config_model(
                level=level, 
                num_events_per_set=num_events_per_set
            ),
            config_dset_eval=config_experiment_deep_sets.get_config_dset(
                level=level, 
                num_events_per_set=num_events_per_set, 
                kind="eval"
            ),
            generate_dsets=True,
        )

Eval

In [5]:
for level in Names_Levels().tuple_:

    for num_events_per_set in Nums_Events_Per_Set().tuple_:

        experiment.evaluate(
            config_model=config_experiment_deep_sets.get_config_model(
                level=level, 
                num_events_per_set=num_events_per_set
            ), 
            config_dset_eval=config_experiment_deep_sets.get_config_dset(
                level=level, 
                num_events_per_set=num_events_per_set, 
                kind="eval"
            ),
            config_dset_eval_sens=config_experiment_deep_sets.get_config_dset(
                level=level, 
                num_events_per_set=num_events_per_set, 
                kind="eval_sens"
            ),
            generate_dsets=False, 
        )

Loaded tensor of shape: torch.Size([2000, 70000, 4]) from: ..\..\state\new_physics\data\processed\sets_unbinned_gen_q2v_loose\70000_eval_sens_features.pt
Loaded tensor of shape: torch.Size([2000]) from: ..\..\state\new_physics\data\processed\sets_unbinned_gen_q2v_loose\70000_eval_sens_labels.pt
Loaded dataset: sets_unbinned
Loaded tensor of shape: torch.Size([2200, 70000, 4]) from: ..\..\state\new_physics\data\processed\sets_unbinned_gen_q2v_loose\70000_eval_features.pt
Loaded tensor of shape: torch.Size([2200]) from: ..\..\state\new_physics\data\processed\sets_unbinned_gen_q2v_loose\70000_eval_labels.pt
Loaded dataset: sets_unbinned


c:\Users\tetha\Desktop\btokstll\logic\scripts\helpers\result\table.py:37: PerformanceWarning: indexing past lexsort depth may impact performance.
  self.table.loc[


Loaded tensor of shape: torch.Size([2000, 24000, 4]) from: ..\..\state\new_physics\data\processed\sets_unbinned_gen_q2v_loose\24000_eval_sens_features.pt
Loaded tensor of shape: torch.Size([2000]) from: ..\..\state\new_physics\data\processed\sets_unbinned_gen_q2v_loose\24000_eval_sens_labels.pt
Loaded dataset: sets_unbinned
Loaded tensor of shape: torch.Size([2200, 24000, 4]) from: ..\..\state\new_physics\data\processed\sets_unbinned_gen_q2v_loose\24000_eval_features.pt
Loaded tensor of shape: torch.Size([2200]) from: ..\..\state\new_physics\data\processed\sets_unbinned_gen_q2v_loose\24000_eval_labels.pt
Loaded dataset: sets_unbinned
Loaded tensor of shape: torch.Size([2000, 6000, 4]) from: ..\..\state\new_physics\data\processed\sets_unbinned_gen_q2v_loose\6000_eval_sens_features.pt
Loaded tensor of shape: torch.Size([2000]) from: ..\..\state\new_physics\data\processed\sets_unbinned_gen_q2v_loose\6000_eval_sens_labels.pt
Loaded dataset: sets_unbinned
Loaded tensor of shape: torch.Size(

Event by event Method

In [6]:
config_experiment_ebe = Config_Experiment_Event_by_Event()

Train

In [3]:
for level in (Names_Levels().detector_and_background,):

    experiment.train(
        config_model=config_experiment_ebe.get_config_model(
            level=level, 
        ),
        config_dset_eval=config_experiment_ebe.get_config_dset(
            level=level, 
            split=Names_Splits().eval_,
        ),
        generate_dsets=True,
    )

Opened ..\..\state\new_physics\data\processed\agg_sig_1_to_20_det.pkl
Loaded aggregated raw signal data file: ..\..\state\new_physics\data\processed\agg_sig_1_to_20_det.pkl
Number of NA values: 
 q_squared          0
costheta_mu      144
costheta_K       670
chi              670
dc9_bin_index      0
dtype: int64
Removed NA rows.
Shuffled dataframe.
Applied cleaning.
Loaded raw bkg file: ..\..\state\new_physics\data\raw\bkg\mu_sideb_generic_charge_train_scaled.pkl
Loaded raw bkg file: ..\..\state\new_physics\data\raw\bkg\mu_sideb_generic_mix_train_scaled.pkl
Number of NA values: 
 q_squared      0
costheta_mu    0
costheta_K     0
chi            0
dtype: int64
Removed NA rows.
Shuffled dataframe.
Applied cleaning.
Number of NA values: 
 q_squared      0
costheta_mu    0
costheta_K     0
chi            0
dtype: int64
Removed NA rows.
Shuffled dataframe.
Applied cleaning.
Generated tensor of shape: torch.Size([9330430, 4]).
Saved as: ..\..\state\new_physics\data\processed\events_binned_de

In [4]:
from helpers.plot.loss_curves import plot_loss_curves

plot_loss_curves(
    config_model=config_experiment_ebe.get_config_model(
        level=Names_Levels().detector_and_background, 
    ),
    config_dset_eval=config_experiment_ebe.get_config_dset(
        level=Names_Levels().detector_and_background, 
        split=Names_Splits().eval_,
    ),
    path_dir=path_dir_plots,
)

Eval

In [7]:
for level in (Names_Levels().generator, Names_Levels().detector):

    for num_events_per_set in Nums_Events_Per_Set().tuple_:

        experiment.evaluate(
            config_model=config_experiment_ebe.get_config_model(
                level=level, 
            ), 
            config_dset_eval=config_experiment_ebe.get_config_dset(
                level=level, 
                split=Names_Splits().eval_,
                num_events_per_set=num_events_per_set, 
            ),
            config_dset_eval_sens=config_experiment_ebe.get_config_dset(
                level=level, 
                split=Names_Splits().eval_,
                sens=True,
                num_events_per_set=num_events_per_set, 
            ),
            generate_dsets=False, 
        )

Loaded tensor of shape: torch.Size([2000, 70000, 4]) from: ..\..\state\new_physics\data\processed\sets_binned_gen_q2v_loose\70000_eval_sens_features.pt
Loaded tensor of shape: torch.Size([2000, 44]) from: ..\..\state\new_physics\data\processed\sets_binned_gen_q2v_loose\70000_eval_sens_labels.pt
Loaded tensor of shape: torch.Size([44]) from: ..\..\state\new_physics\data\processed\sets_binned_gen_q2v_loose\70000_eval_sens_bin_map.pt
Loaded dataset: sets_binned
Loaded tensor of shape: torch.Size([2200, 70000, 4]) from: ..\..\state\new_physics\data\processed\sets_binned_gen_q2v_loose\70000_eval_features.pt
Loaded tensor of shape: torch.Size([2200, 44]) from: ..\..\state\new_physics\data\processed\sets_binned_gen_q2v_loose\70000_eval_labels.pt
Loaded tensor of shape: torch.Size([44]) from: ..\..\state\new_physics\data\processed\sets_binned_gen_q2v_loose\70000_eval_bin_map.pt
Loaded dataset: sets_binned


c:\Users\tetha\Desktop\btokstll\logic\scripts\helpers\result\table.py:37: PerformanceWarning: indexing past lexsort depth may impact performance.
  self.table.loc[


Loaded tensor of shape: torch.Size([2000, 24000, 4]) from: ..\..\state\new_physics\data\processed\sets_binned_gen_q2v_loose\24000_eval_sens_features.pt
Loaded tensor of shape: torch.Size([2000, 44]) from: ..\..\state\new_physics\data\processed\sets_binned_gen_q2v_loose\24000_eval_sens_labels.pt
Loaded tensor of shape: torch.Size([44]) from: ..\..\state\new_physics\data\processed\sets_binned_gen_q2v_loose\24000_eval_sens_bin_map.pt
Loaded dataset: sets_binned
Loaded tensor of shape: torch.Size([2200, 24000, 4]) from: ..\..\state\new_physics\data\processed\sets_binned_gen_q2v_loose\24000_eval_features.pt
Loaded tensor of shape: torch.Size([2200, 44]) from: ..\..\state\new_physics\data\processed\sets_binned_gen_q2v_loose\24000_eval_labels.pt
Loaded tensor of shape: torch.Size([44]) from: ..\..\state\new_physics\data\processed\sets_binned_gen_q2v_loose\24000_eval_bin_map.pt
Loaded dataset: sets_binned
Loaded tensor of shape: torch.Size([2000, 6000, 4]) from: ..\..\state\new_physics\data\pr

In [ ]:
experiment.table_summary.table


In [ ]:
import pandas

split = "train"

bounds_trials = (
    (1, 20) if split == "eval" 
    else (21, 40) if split == "train" 
    else None
)

df = pandas.concat(
    [
        pandas.read_pickle(f"../../state/new_physics/data/processed/agg_sig_{bounds_trials[0]}_to_{bounds_trials[1]}_det.pkl"),
        pandas.read_pickle(f"../../state/new_physics/data/raw/bkg/mu_sideb_generic_charge_{split}_scaled.pkl"),
        pandas.read_pickle(f"../../state/new_physics/data/raw/bkg/mu_sideb_generic_mix_{split}_scaled.pkl"),
    ]
)

df[(df["q_squared"] < 20) & (df["q_squared"] > 0)].std()

In [8]:
from math import sqrt
import pathlib

import matplotlib.pyplot as plt
import pandas

from helpers.plot.util import add_plot_note
from helpers.result.constants import Names_Kinds_Items


def plot(table):

    y_lims = [
        (0, 0.58), 
        (0, 0.62), 
        (0, 0.65), 
        (-1.2, -0.65), 
        (-0.4, 0.4)
    ]

    num_sets_nominal = 50
    num_sets_sens = 2000

    q_squared_veto = Names_q_Squared_Vetos().loose

    names_models = (Names_Models().cnn, Names_Models().deep_sets, Names_Models().ebe)

    markers = ["o", "v", "s"]
    colors = ["#999999", "#377eb8", "#a65628"]

    linestyles = ["-", "--", "-."]

    for col, y_lim in zip(
        table.columns, 
        y_lims
    ):

        _, ax = plt.subplots()
        
        for name_model, marker, color in zip(
            names_models, 
            markers, 
            colors
        ):
            
            for level, linestyle in zip(
                Names_Levels().tuple_, 
                linestyles
            ):
                
                if (
                    (name_model == Names_Models().ebe) 
                    and (level == Names_Levels().detector_and_background)
                ):
                    continue

                y = table.loc[
                    pandas.IndexSlice[
                        level, 
                        q_squared_veto, 
                        name_model,
                    ], 
                    col,
                ]

                x = y.index

                fmt = linestyle + marker
                
                ax.plot(
                    x, 
                    y, 
                    fmt, 
                    label=f"{name_model}, {level}", 
                    c=color, 
                    markersize=5, 
                    alpha=.8
                    )
                
                if (
                    (col==Names_Kinds_Items().np_bias) 
                    or (col==Names_Kinds_Items().np_mean)
                ):
                
                    errors = (
                        table.loc[
                            pandas.IndexSlice[
                                level, 
                                q_squared_veto, 
                                name_model,
                            ], 
                            Names_Kinds_Items().np_std,
                        ] 
                        / sqrt(num_sets_sens)
                    )
                
                    ax.errorbar(
                        x=x, 
                        y=y, 
                        yerr=errors, 
                        fmt='none', 
                        elinewidth=0.5, 
                        capsize=0.5, 
                        color="black",
                    )

            ax.set_ylim(y_lim)
            ax.set_ylabel(f"{col}")
            ax.set_xlabel("Number of events / set")
            ax.legend(ncols=2, markerscale=0.5, numpoints=1)

            if (
                (col==Names_Kinds_Items().np_bias) 
                or (col==Names_Kinds_Items().np_mean) 
                or (col==Names_Kinds_Items().np_std)
            ):
                add_plot_note(
                    ax, 
                    f"Num boots.: {num_sets_sens}",
                )
            
            else: 
                add_plot_note(
                    ax, 
                    f"Num boots./label: {num_sets_nominal}",
                )

        file_name = f"comp_{col}.png"
        file_path = pathlib.Path(path_dir_plots).joinpath(file_name)

        plt.savefig(file_path, bbox_inches="tight")
        plt.close()

plot(experiment.table_summary.table)

C:\Users\tetha\AppData\Local\Temp\ipykernel_8836\2133017417.py:57: PerformanceWarning: indexing past lexsort depth may impact performance.
  y = table.loc[
C:\Users\tetha\AppData\Local\Temp\ipykernel_8836\2133017417.py:86: PerformanceWarning: indexing past lexsort depth may impact performance.
  table.loc[


In [ ]:
import pandas

df_charge = pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_charge_whole.pkl")

df_mix = pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_mix_whole.pkl")


In [ ]:
def split(df):
    ind_half = int(len(df)/2)
    df_train = df[:ind_half].copy()
    df_eval = df[ind_half:].copy()
    return df_train, df_eval

df_charge_train, df_charge_eval = split(df_charge.loc["det"][list(Names_Variables().tuple_)])

df_mix_train, df_mix_eval = split(df_mix.loc["det"][list(Names_Variables().tuple_)])


In [ ]:
pandas.to_pickle(df_charge_train, "../../state/new_physics/data/raw/bkg/mu_sideb_generic_charge_train.pkl")

In [ ]:
pandas.to_pickle(df_charge_eval, "../../state/new_physics/data/raw/bkg/mu_sideb_generic_charge_eval.pkl")


In [ ]:
pandas.to_pickle(df_mix_train, "../../state/new_physics/data/raw/bkg/mu_sideb_generic_mix_train.pkl")


In [ ]:
pandas.to_pickle(df_mix_eval, "../../state/new_physics/data/raw/bkg/mu_sideb_generic_mix_eval.pkl")


In [ ]:
pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_mix_train.pkl")

In [ ]:
pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_mix_eval.pkl")


In [ ]:
pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_charge_train.pkl")

In [ ]:
pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_charge_eval.pkl")

In [ ]:
import pandas
from helpers.data.dset.preproc import apply_q_squared_veto


df_bkg_charge = pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_charge_train.pkl")
df_bkg_mix = pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_mix_train.pkl")

df_bkg = pandas.concat([df_bkg_charge, df_bkg_mix])

df_agg = pandas.read_pickle("../../state/new_physics/data/processed/agg_sig_1_to_20_det.pkl")

df_combo = pandas.concat([df_bkg[list(Names_Variables().tuple_)], df_agg[list(Names_Variables().tuple_)]])

df_combo_q2v_tight = apply_q_squared_veto(df_combo, "tight")

df_combo_q2v_tight.mean()



In [ ]:
df_agg

In [ ]:
import pandas

df = pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_mix_whole.pkl").loc["det"]
df[df["isSignal"]==0]